In [1]:
import pandas as pd
import gensim
from sklearn.metrics.pairwise import cosine_similarity
import numpy as np
import spacy
%run calculateSentence.ipynb
import preprocessing
import tfidf
import scipy
pd.set_option('max_colwidth', 5000)

In [2]:
#Importazione DB

#Data on Stack Overflow DB already preprocessed
data_stack = pd.read_csv('DB/Preprocessed_data.csv')

#data = data[data['processed_title'].notna()]

#Data on Ontology DB already preprocessed
data_onto = pd.read_csv('DB/Preprocessed_ontology.csv')

#Matrix for stack tfidf processing 
#tfidf_stack = scipy.sparse.load_npz('DB/tfidf_stack_matrix.npz')
stack_corpus = data_stack.post_corpus

#Matrix for ontology tfidf processing
#tfidf_onto = scipy.sparse.load_npz('DB/tfidf_onto_matrix.npz')
onto_corpus = data_onto.description_processed

In [3]:
#Importazione WordEmbeddings created in file 'train_word2vec'

word_2_vec_model = gensim.models.word2vec.Word2Vec.load('DB/word2vec_trained.bin')
word_2_vec_model

In [4]:
#Fucntion to clear raw txt 
def normalize(txt):
    phrase = preprocessing.clear_text(txt)
    return phrase

In [5]:
#Fucntion to calculate cosine similarity between user raw txt and stack overflow processed title with word to vec  
def w2v_stack_result(txt):
    stringSearch = normalize(txt)

    #print(stringSearch)

    numberResult = 5

    #Viene vettorizzata la query dell'utente

    vectorSearch = np.array([questionToVec(stringSearch, word_2_vec_model)])
    
    #Carico il file con gli embeddings dei titoli sui quali effettuare la corrispondenza
    allTitleEmb = pd.read_csv('DB/titleEmbeddings.csv').values

    #Calcolo della similarità del coseno for le query e tutti i titoli
    similarityCosine = pd.Series(cosine_similarity(vectorSearch, allTitleEmb)[0])
    
    #Personalizzazione della misura di similarità
    similarityCosine = similarityCosine*(1+0.4*data_staack.score)
    
    #Lista che conterrà coppie i,j con i indice e j score di similarità
    results = []
    
    #Una volta calcolati gli score di similarità rendo i 10 più alti
    for i,similiarity_score in similarityCosine.nlargest(numberResult).iteritems():
        #print(i, data.Body[i])
        #Filtro i tag da rendere come risultato, eliminando quelli che sono già presenti nella string search per facilitare prossimi passaggi 
        tags = data_stack.iloc[[i]].tags.item().split('|')
        #print(tags)
        for t in tags:
            if t in stringSearch:
                tags.remove(t)
                
        stack_id = data_stack.iloc[[i]].id.values[0]
        stack_title = data_stack.iloc[[i]].title.values[0]
        
        results.append(tuple([stack_id, similiarity_score, tags, stack_title ]))
        
    return results

In [6]:
def w2v_ontology_result(txt):
    stringSearch = normalize(txt)

    #print(stringSearch)

    numberResult = 3

    #Viene vettorizzata la query dell'utente

    vectorSearch = np.array([questionToVec(stringSearch, word_2_vec_model)])
    
    #Carico il file con gli embeddings dei titoli sui quali effettuare la corrispondenza
    allFuncEmb = pd.read_csv('DB/functionEmbeddings.csv').values

    #Calcolo della similarità del coseno for le query e tutti i titoli

    similarityCosine = pd.Series(cosine_similarity(vectorSearch, allFuncEmb)[0])
    
    #Lista che conterrà coppie i,j con i indice e j score di similarità
    results = []
    
      
    #Una volta calcolati gli score di similarità rendo i 10 più alti
    for i,similiarity_score in similarityCosine.nlargest(numberResult).iteritems():
        #print(i, data.Body[i])
        #Filtro i tag da rendere come risultato, eliminando quelli che sono già presenti nella string search per facilitare prossimi passaggi 
        description = data_onto.iloc[[i]].description.item()
        function = data_onto.iloc[[i]].function.item()
                    
        results.append(tuple([function,description, similiarity_score]))
      
    return results

In [7]:
def tfidf_stack_result(txt):
    stringSearch = normalize(txt)
    related_docs_indices = tfidf.get_results(stack_corpus, stringSearch)
    return related_docs_indices

In [8]:
indices = tfidf_stack_result('problem loading csv file')
print(indices)

[38348 24245  9765 33960]


In [9]:
data.iloc[[38348]]

,id,title,tags,score,post_corpus,questions,processed_title
38348,68513445,Append Pandas dataframe to top of csv file without loading csv file content,pandas|dataframe|csv|export-to-csv,-0.000527,append pandas dataframe top csv file without loading csv file content appending content dataframe csv file csv data appended bottom possible append data top without loading content csv file issues large file sizes python memory issues easier way save df csv use combining terminal linux winodws,"Append Pandas dataframe to top of csv file without loading csv file content When appending content of dataframe to csv file with &quot;to_csv&quot; the data is appended to the bottom. Is it possible to append the data into the top, without loading all the content of the csv file?",append pandas dataframe top csv file without loading csv file content
